#Drinks Classifier

In [ ]:
from skimage.feature import hog
#from skimage.io import imread
import joblib,glob,os,cv2

from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

from google.colab import files

##Importing Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Preparation

In [ ]:
X = []
Y = []

pepsi_path = '/content/drive/MyDrive/Drinks Classifier/pepsi_images'
cocacola_path = '/content/drive/MyDrive/Drinks Classifier/coca_cola_images'
heineken_path = '/content/drive/MyDrive/Drinks Classifier/heineken_beer_images'


In [ ]:
#coca cola

for filename in glob.glob(os.path.join(cocacola_path,"*.jpg")):
    fd = cv2.imread(filename,0)
    fd = cv2.resize(fd,(64,128))
    #fd = cv2.cvtColor(fd, cv2.COLOR_BGR2GRAY)
    print(fd)
    X.append(fd)
    Y.append(0)


In [ ]:
#pepsi

for filename in glob.glob(os.path.join(pepsi_path,"*.jpg")):
    fd = cv2.imread(filename,0)
    fd = cv2.resize(fd,(64,128))

    print(fd)
    X.append(fd)
    Y.append(1)

In [ ]:
# heineken
for filename in glob.glob(os.path.join(heineken_path,"*.jpg")):
    fd = cv2.imread(filename,0)
    fd = cv2.resize(fd,(64,128))

    print(fd)
    X.append(fd)
    Y.append(2)

In [ ]:
len(X)

1676

##Machine Learning

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train_reshaped = np.array(X_train).reshape(len(X_train), -1)  # Reshape images to 2D arrays
y_train_reshaped = np.array(y_train).reshape(len(y_train), -1)  # Reshape images to 2D arrays


In [ ]:
X_test_reshaped = np.array(X_test).reshape(len(X_test), -1)
y_test_reshaped = np.array(y_test).reshape(len(y_test), -1)

In [ ]:
# Training the SVC model with gamma and C found in previous step

#classifier = svm.SVC(gamma=0.00728932024638, C=2.82842712475)
classifier = svm.SVC(gamma=0.001, C=2.0)
classifier.fit(X_train_reshaped, y_train_reshaped)



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=2.0, gamma=0.001)

In [ ]:
# Calculating the training accuracy (to measure the bias)
train_accuracy = classifier.score(X_train_reshaped, y_train_reshaped)
print (f"Training Accuracy: {train_accuracy*100:.3f}%")

# Getting predictions
predictions = classifier.predict(X_test_reshaped)

Training Accuracy: 100.000%


In [ ]:
model = LinearSVC()
model.fit(X_train_reshaped,y_train_reshaped)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [ ]:
# predict
y_pred = model.predict(X_test_reshaped)

In [ ]:
# confusion matrix and accuracy

from sklearn import metrics
from sklearn.metrics import classification_report

print(f"Classification report for classifier {model}:\n"
      f"{metrics.classification_report(y_test_reshaped, y_pred)}\n")

Classification report for classifier LinearSVC():
              precision    recall  f1-score   support

           0       0.52      0.50      0.51        80
           1       0.63      0.76      0.69       123
           2       0.81      0.66      0.73       133

    accuracy                           0.66       336
   macro avg       0.65      0.64      0.64       336
weighted avg       0.67      0.66      0.66       336




#Deep Learning

In [ ]:
# Import of keras model and hidden layers for our convolutional network
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten

In [ ]:
# Construction of model
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(64, 128, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
# Turn X and y into np.array to speed up train_test_split
X = np.array(X, dtype="uint8")
X = X.reshape(len(X), 64, 128, 1) # Needed to reshape so CNN knows it's different images
Y = np.array(Y)

print("Images loaded: ", len(X))
print("Labels loaded: ", len(Y))

Images loaded:  1676
Labels loaded:  1676


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
# Configures the model for training
model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss='sparse_categorical_crossentropy', # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 60, 124, 32)       832       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 30, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 60, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 28, 64)        36928     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 6, 14, 64)       

In [ ]:
X_train_reshaped = np.array(X_train_reshaped).reshape(-1, 64, 128, 1)  # Reshape to (batch_size, height, width, channels)
X_test_reshaped = np.array(X_test_reshaped).reshape(-1, 64, 128, 1)  # Reshape to (batch_size, height, width, channels

In [ ]:
y_train_reshaped =np.array(y_train)  # Reshape to (batch_size, height, width, channels)
y_test_reshaped =np.array(y_test)   # Reshape to (batch_size, height, width, channels

In [ ]:
import tensorflow as tf


In [ ]:
y_train_reshaped.shape

(1340,)

In [ ]:
X_train_reshaped.shape

(1340, 64, 128, 1)

In [ ]:
y_test_reshaped.shape

(336,)

In [ ]:
X_test_reshaped.shape

(336, 64, 128, 1)

In [ ]:
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=2, validation_data=(X_test, y_test))

Epoch 1/5
21/21 - 15s - loss: 11.9055 - accuracy: 0.4306 - val_loss: 0.9196 - val_accuracy: 0.5595 - 15s/epoch - 711ms/step
Epoch 2/5
21/21 - 13s - loss: 0.7819 - accuracy: 0.6493 - val_loss: 0.7729 - val_accuracy: 0.6548 - 13s/epoch - 641ms/step
Epoch 3/5
21/21 - 13s - loss: 0.5983 - accuracy: 0.7433 - val_loss: 0.6245 - val_accuracy: 0.6964 - 13s/epoch - 619ms/step
Epoch 4/5
21/21 - 13s - loss: 0.4746 - accuracy: 0.8119 - val_loss: 0.5361 - val_accuracy: 0.7560 - 13s/epoch - 615ms/step
Epoch 5/5
21/21 - 14s - loss: 0.3527 - accuracy: 0.8619 - val_loss: 0.4581 - val_accuracy: 0.8125 - 14s/epoch - 644ms/step


In [ ]:
# Save entire model to a HDF5 file
model.save('drinks_classifier.h5')

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy: {:2.2f}%'.format(test_acc*100))

11/11 [==============================] - 2s 141ms/step - loss: 0.4581 - accuracy: 0.8125
Test accuracy: 81.25%


In [ ]:
predictions = model.predict(X_test) # Make predictions towards the test set

188/188 [==============================] - 64s 340ms/step


In [ ]:
np.argmax(predictions[200]), y_test[200] # If same, got it right

(0, 0)

In [ ]:
# Function to plot images and labels for validation purposes
def validate_9_images(predictions_array, true_label_array, img_array):
  # Array for pretty printing and then figure size
  class_names = ["down", "palm", "l", "fist", "fist_moved", "thumb", "index", "ok", "palm_moved", "c"]
  plt.figure(figsize=(15,5))

  for i in range(1, 10):
    # Just assigning variables
    prediction = predictions_array[i+30]
    true_label = true_label_array[i+30]
    img = img_array[i+30]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # Plot in a good way
    plt.subplot(3,3,i)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(prediction) # Get index of the predicted label from prediction

    # Change color of title based on good prediction or not
    if predicted_label == true_label:
      color = 'blue'
    else:
      color = 'red'

    plt.xlabel("Predicted: {} {:2.0f}% (True: {})".format(class_names[predicted_label],
                                  100*np.max(prediction),
                                  class_names[true_label]),
                                  color=color)
  plt.show()